
**Created by:**

__Viktor Varga__

<br>

<img src="https://docs.google.com/uc?export=download&id=1q8cQBQKSLqS3PirWEmtQyObewgHOVISl" style="display:inline-block" width='40%'>
<hr>

# Python tutorial - Iterátorok és generátorok

## Iterátorok

A lusta kiértékelés (lazy evaluation) elősegítésének egyik eszköze az iterátor. A Python nyelv régebbi, 2-es verziójában sok beépített művelet egy, vagy több listával tért vissza. Ez nem ideális abban az esetben, ha nagyméretű listák generálása szükséges, azonban annak csak első néhány elemével dolgozunk tovább. A Python 3-as verziójában az ilyen, beépített műveletek többsége iterátort ad vissza. Az iterátor a soron következő elemet csak abban az esetben állítja elő, ha mi arra igényt tartunk.



Python 3-ban iterálható típust olyan osztály tud megvalósítani, aminek van `__iter__()` és `__next__()` tagfüggvénye. Előbbi tipikusan a `self`-et, azaz magát az iterátor típus példányát adja vissza,  míg utóbbi a soron következő értéket.

Alább egy új iterátort implementálunk, ami egész számokon csúszó ablak (sliding window) szerű intervallumokat ad vissza.

In [ ]:
# custom iterator class: returning sliding windows

class SlidingWindowIterator:

  def __init__(self, n, window_len):
    self.n = n
    self.window_len = window_len
    self.current = 0

  def __iter__(self):
    return self

  def __next__(self):
    if self.current+self.window_len-1 < self.n:
      old_current = self.current
      self.current += 1
      return list(range(old_current, old_current+self.window_len))
    else:
      raise StopIteration()


Példányosítsuk az új iterátor osztályunkat!

In [ ]:
print("The sliding windows:")
it = SlidingWindowIterator(5,3)  # windows with a length of 3 from 0 to 5
for item in it:
  print("    ", item)

The sliding windows:
     [0, 1, 2]
     [1, 2, 3]
     [2, 3, 4]


In [ ]:
print("Let's try to continue the iteration!")
for item in it:
  print("    ", item)

Let's try to continue the iteration!


Amint látjuk, az első for ciklus végigiterált az iterátor példányon, elhasználva azt. A következő for ciklus emiatt már nem tud kiolvasni belőle semmit. A for ciklus az iterátor `__next__()` tagfüggvényét hívja újra és újra, egészen addig, míg az nem dob egy `StopIteration` kivételt.

Példányosítsuk újra az iterátor osztályt és lépegessünk rajta a `__next__()` tagfüggvény hívogatásával. A beépített `next()` függvény ugyanezt teszi: az argumentumként adott iterátor `__next__()` tagfüggvényét hívja meg.

In [ ]:
# new iterator
it = SlidingWindowIterator(5,3)
try:
  print("The first element: ", it.__next__())
  print("The second element: ", it.__next__())
  print("The third element: ", next(it))   # Python builtin next() function calls __next__()
  print("The fourth element: ", next(it))
except StopIteration:
  print("StopIteration was raised: reached end of iterator.")


The first element:  [0, 1, 2]
The second element:  [1, 2, 3]
The third element:  [2, 3, 4]
StopIteration was raised: reached end of iterator.


Sok Python 3 beépített függvény tér vissza iterátorokat használó objektumokkal. Ilyen például a `range` osztály is, amit a `range()` beépített függvény ad vissza. Maga a `range` osztály nem iterátor, de megvalósítja az `__iter__()` tagfüggvényt, ami egy iterátort ad vissza, ami már iterálható.

In [ ]:
range_obj = range(5, 10, 2)
print("The type of range function's return value is ", type(range_obj))
print("Iterating through the range object's iterator:")
for item in range_obj:
  print("    ", item)

# Calling next(range_obj) would result in: "TypeError: 'range' object is not an iterator"

# Instead, get the range object's iterator
range_obj_it = range_obj.__iter__()
print("\nThe type of range object's iterator ", type(range_obj_it))

print("Now we can call iterate through the iterator by calling next() multiple times:")
try:
  print("    The first element: ", range_obj_it.__next__())
  print("    The second element: ", range_obj_it.__next__())
  print("    The third element: ", next(range_obj_it)) # Python builtin next() function calls __next__()
  print("    The fourth element: ", next(range_obj_it))
except StopIteration:
  print("    StopIteration was raised: reached end of iterator.")


The type of range function's return value is  <class 'range'>
Iterating through the range object's iterator:
     5
     7
     9

The type of range object's iterator  <class 'range_iterator'>
Now we can call iterate through the iterator by calling next() multiple times:
    The first element:  5
    The second element:  7
    The third element:  9
    StopIteration was raised: reached end of iterator.


Azaz, a for ciklusban történő iterációkor az `in` kulcsszó utáni objektum `__iter__()` tagfüggvénye hívódik először. Ha az objektum maga egy iterátor már, az `__iter__()` tagfüggvénye a `self`-et, azaz önmagát adja vissza. A `range` típusú objektum önmaga nem iterátor, de az `__iter__()` meghívására visszad egy `range_iterator` iterátor objektumot, ami már iterálható.

Miért jó ez? Alább láthatjuk a különbséget:

In [ ]:
range_obj = range(3)
range_obj_it = range(3).__iter__()

try:
    print("Starting iteration#1...")
    for item in range_obj:
        print("    ", item)
    print("Starting iteration#2...")
    for item in range_obj:
        print("    ", item)
except StopIteration:
    print("    StopIteration was raised: reached end of iterator.")

print("\nA range object can be used for multiple iterations. Now let's try the iterator itself:")

try:
    print("Starting iteration#1...")
    for item in range_obj_it:
        print("    ", item)
    print("Starting iteration#2...")
    for item in range_obj_it:
        print("    ", item)
except StopIteration:
    print("    StopIteration was raised: reached end of iterator.")

print("\nThe iterator itself expires after it is iterated through.")

Starting iteration#1...
     0
     1
     2
Starting iteration#2...
     0
     1
     2

A range object can be used for multiple iterations. Now let's try the iterator itself:
Starting iteration#1...
     0
     1
     2
Starting iteration#2...

The iterator itself expires after it is iterated through.


Tehát, a `range` típusú objektum többször használható, míg az általa visszadott iterátor csak egyszer.

Hasonló objektumot kapunk, ha lekérdezzük egy `dict` adatszerkezet kulcshalmazát:

In [ ]:
my_dict = {1:"egy", 2:"ketto", 3:"harom"}

my_keys = my_dict.keys()
print("The key set: ", my_keys)
print("The type of the key set: ", type(my_keys))
print("The type of the key set's iterator: ", type(my_keys.__iter__()))

print("Starting iteration#1...")
for item in my_keys:
    print("    ", item)
print("Starting iteration#2...")
for item in my_keys:
    print("    ", item)

The key set:  dict_keys([1, 2, 3])
The type of the key set:  <class 'dict_keys'>
The type of the key set's iterator:  <class 'dict_keyiterator'>
Starting iteration#1...
     1
     2
     3
Starting iteration#2...
     1
     2
     3


## Generátorok

Iterátor megvalósítása rövidebben. A generátor `yield` utasítása a `return` utasításhoz hasonló, de a függvény miután visszaadta a kifejezésben szereplő értéket, felfüggeszti futását és a következő iterációs lépéskor folytatja azt, egészen addig amíg egy újabb `yield` utasításba nem ütközik. Amíg a generátor felfüggesztett állapotban várakozik, a lokális változók értékeit megtartja. A generátor futása csak akkor ér véget, ha egy `return` utasításhoz, vagy a generátor kódjának végéhez ér.

A generátor az iterátorokhoz hasonlóan léptethető, a `generator` osztály megvalósítja az `__iter__()` és `__next__()` tagfüggvényeket is.

A notebook elején definiált iterátor osztályt, a `SlidingWindowIterator` osztályt most valósítsuk meg generátorként!

In [ ]:
# generator: returning sliding windows

def sliding_window_generator(n, window_len):
  for idx in range(0,n-window_len+1):
    print("    Upcoming yield!")   # this is only here for demonstration purposes
    yield list(range(idx, idx+window_len))

it = sliding_window_generator(5,3)
print("The type of the generator: ", type(it))
print("The generator returns self on __iter__() call: ", type(it.__iter__()))
print("  ... in short, it is an iterator.")

print("Now let's iterate through it: ")
for item in it:
    print("    ", item)

print("\nLet's try to do one more iteration step on it:")
try:
    next(it)
except StopIteration:
    print("    StopIteration was raised: reached end of iterator.")

print("\nAs we see, generators are like iterators, when they reach the end, they expire...")

it = sliding_window_generator(5,3)
print("But we can instantiate a new one: ", next(it), " ...")


The type of the generator:  <class 'generator'>
The generator returns self on __iter__() call:  <class 'generator'>
  ... in short, it is an iterator.
Now let's iterate through it: 
    Upcoming yield!
     [0, 1, 2]
    Upcoming yield!
     [1, 2, 3]
    Upcoming yield!
     [2, 3, 4]

Let's try to do one more iteration step on it:
    StopIteration was raised: reached end of iterator.

As we see, generators are like iterators, when they reach the end, they expire...
    Upcoming yield!
But we can instantiate a new one:  [0, 1, 2]  ...


### Generátor kifejezés

List/set/dict comprehension-höz hasonlóan, egy generátor is sokszor felírható rövidebb alakban, generátor kifejezésként (generator expression). Ha elhasználtuk az iterátort amit a generátor kifejezés adott, újra kell definiálnunk. Az `itertools` csomag `tee()` függvénye használható iterátor objektumok másolására, körültekintéssel (lásd dokumentáció).

In [ ]:
window_len = 3
n = 5

sliding_window_generator = (list(range(i, i+window_len)) for i in range(n-window_len+1))

print("\nLet's iterate through it:")
for item in sliding_window_generator:
  print("    ", item)

# to reuse the generator expression, we must define it again


Let's iterate through it:
     [0, 1, 2]
     [1, 2, 3]
     [2, 3, 4]
